# Image classification ? fastai (data + train + eval)


Commentaire: imports fastai et modules internes


In [2]:
# Imports fastai
import sys
from pathlib import Path
sys.path.append(str(Path('../..').resolve()))
from fastai.vision.all import vision_learner, resnet34, accuracy, F1Score, RocAuc, ClassificationInterpretation, SaveModelCallback, EarlyStoppingCallback
import matplotlib.pyplot as plt
from model.image.fastai.data import DataConfig, build_dls
from model.image.fastai.augmentations import NoiseConfig


Commentaire: configuration data + toggles bruit/masque


In [4]:
# Configuration data
from pathlib import Path

class_names = ['Chao', 'Milho', 'Ervas', 'Milho_ervas']
candidates = [
    Path('../data/ImagensTCCRotuladas/Treino'),
    Path('../../data/ImagensTCCRotuladas/Treino'),
    Path('data/ImagensTCCRotuladas/Treino'),
]
base_dir = next((c.resolve() for c in candidates if c.exists()), None)
if base_dir is None:
    raise FileNotFoundError('Aucun dossier Treino trouv?; v?rifiez le chemin data/ImagensTCCRotuladas')

cfg = DataConfig(
    data_dir=base_dir,
    img_size=224,
    bs=32,
    valid_pct=0.2,
    use_noise=False,
    use_mask=False,
    per_class_limit=2000,
    class_names=class_names,
)
print('Base dir:', base_dir)
print('Classes:', cfg.class_names)


Base dir: C:\Users\lmanuelli\Projet\TP_final\data\ImagensTCCRotuladas\Treino
Classes: ['Chao', 'Milho', 'Ervas', 'Milho_ervas']


Commentaire: construis les DataLoaders (changer use_noise/use_mask pour tester l'augmentation custom)


In [5]:
# DataLoaders
classes_mode = 4
dls = build_dls(cfg, classes_mode=classes_mode)
print('Nombre de classes:', dls.c)
print('Vocab:', dls.vocab)


Nombre de classes: 4
Vocab: ['Chao', 'Ervas', 'Milho', 'Milho_ervas']


Commentaire: entra?nement fastai (fine_tune)


In [6]:
# Entra?nement fastai
learn = vision_learner(
    dls,
    resnet34,
    metrics=[accuracy, F1Score(average='macro'), RocAuc()],
    pretrained=True,
)
learn.fine_tune(2, base_lr=1e-3)


epoch,train_loss,valid_loss,accuracy,f1_score,roc_auc_score,time


ValueError: Exception occured in `Recorder` when calling event `after_validate`:
	Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].

Commentaire: courbes + matrice de confusion


In [ ]:
# Courbes et confusion matrix
learn.recorder.plot_loss()
plt.show()
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(figsize=(5,5))
plt.show()


Commentaire: rapport par classe


In [ ]:
# Rapport per-class (val)
from sklearn.metrics import classification_report
preds, targs = learn.get_preds()
preds_idx = preds.argmax(dim=1)
print(classification_report(targs, preds_idx, target_names=dls.vocab, digits=3))


Commentaire: sauvegarde des artefacts fastai dans model/registry


In [ ]:
# Sauvegarde fastai dans model/registry
from datetime import datetime
import json
run_id = datetime.utcnow().strftime('%Y%m%d_%H%M%S')
run_dir = Path('../model/registry') / f'image_fastai_{run_id}_notebook'
run_dir.mkdir(parents=True, exist_ok=True)
learn.export(run_dir / 'model.pkl')
payload = {
    'classes': list(dls.vocab),
    'img_size': cfg.img_size,
    'bs': cfg.bs,
    'valid_pct': cfg.valid_pct,
    'use_noise': cfg.use_noise,
    'use_mask': cfg.use_mask,
    'per_class_limit': cfg.per_class_limit,
}
(run_dir / 'summary.json').write_text(json.dumps(payload, indent=2), encoding='utf-8')
print('Artifacts saved to', run_dir)


Commentaire: ?valuation compl?te sur le test (apr?s entra?nement).


In [ ]:
# ?valuation compl?te sur test (fastai)
from sklearn.metrics import classification_report
test_dir = Path('../data/ImagensTCCRotuladas/Teste').resolve()
test_dl = dls.test_dl(list(test_dir.rglob('*.jpg')))
preds, targs = learn.get_preds(dl=test_dl)
preds_idx = preds.argmax(dim=1)
print(classification_report(targs, preds_idx, target_names=dls.vocab, digits=3))
